<a href="https://colab.research.google.com/github/lunaB/tutor_resource/blob/main/19%ED%9A%8C%EC%B0%A8_%EA%B3%BC%EC%99%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c titanic

  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 7.11MB/s]
  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 56.4MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 2.88MB/s]


In [74]:
import pandas as pd

train_df = pd.read_csv('train.csv')
train_df = train_df[['Survived','Pclass','Sex','Fare']]

test_df = pd.read_csv('test.csv')
test_df = test_df[['Pclass','Sex','Fare']]

In [34]:
import numpy as np
a = np.array([[1,3,1],[2,1,1],[2,1,1],[2,1,1],[2,1,1],[2,1,1],[2,1,1],[2,1,1]])
print(a.shape)
a.reshape((-1, 6))

(8, 3)


array([[1, 3, 1, 2, 1, 1],
       [2, 1, 1, 2, 1, 1],
       [2, 1, 1, 2, 1, 1],
       [2, 1, 1, 2, 1, 1]])

In [42]:
a = np.array([1,2,3,32,1,2,1,2,3,2,12,1])
print(a.shape)
a.reshape((-1,1))

(12,)


array([[ 1],
       [ 2],
       [ 3],
       [32],
       [ 1],
       [ 2],
       [ 1],
       [ 2],
       [ 3],
       [ 2],
       [12],
       [ 1]])

In [56]:
train_df.loc[:,'Sex'] = train_df.loc[:,'Sex'].str.replace('female', '0').replace('male', '1')
train_df['Sex'] = train_df['Sex'].astype(np.int32)
train_df[['Pclass','Sex','Fare']].values

array([[ 3.    ,  1.    ,  7.25  ],
       [ 1.    ,  0.    , 71.2833],
       [ 3.    ,  0.    ,  7.925 ],
       ...,
       [ 3.    ,  0.    , 23.45  ],
       [ 1.    ,  1.    , 30.    ],
       [ 3.    ,  1.    ,  7.75  ]])

In [72]:
import torch 
import torch.nn as nn
import torch.optim as optim

x_train = torch.FloatTensor(train_df[['Pclass','Sex','Fare']].values)
y_train = torch.FloatTensor(train_df['Survived'].values.reshape((-1,1)))

model = nn.Linear(3, 1)

optimizer = optim.Adam(model.parameters(), lr=0.01)
loss = nn.MSELoss()

for i in range(100):
    y = model(x_train)
    
    cost = loss(y, y_train)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    if i%10 == 0:
        print(i, cost.item())
        print('%.2f%%'%((model(x_train).round().abs() == y_train).float().mean()*100))

0 1.5671764612197876
32.88%
10 0.8275147676467896
41.41%
20 0.4315701723098755
65.77%
30 0.283161461353302
65.66%
40 0.24939650297164917
64.76%
50 0.2408401221036911
62.40%
60 0.22852502763271332
66.89%
70 0.21602536737918854
68.35%
80 0.2062598019838333
69.70%
90 0.19821462035179138
69.70%


In [75]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  418 non-null    int64  
 1   Sex     418 non-null    object 
 2   Fare    417 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 9.9+ KB


In [84]:
test_df.head()
test_df['Fare'].fillna(test_df['Fare'].mean(), inplace=True)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  418 non-null    int64  
 1   Sex     418 non-null    object 
 2   Fare    418 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 9.9+ KB


In [85]:
test_df.loc[:,'Sex'] = test_df.loc[:,'Sex'].str.replace('female', '0').replace('male', '1')
test_df['Sex'] = test_df['Sex'].astype(np.int32)
test_df[['Pclass','Sex','Fare']].values
x_test = torch.FloatTensor(test_df[['Pclass','Sex','Fare']].values)

In [91]:
submit_df = pd.read_csv('gender_submission.csv')
submit_df.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [93]:
submit_df['Survived'] = model(x_test).round().int().numpy()
submit_df

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [96]:
submit_df.to_csv('submission.csv', index=False)

In [97]:
!kaggle competitions submit -c titanic -f submission.csv -m "과외용 ㅎ"

100% 2.77k/2.77k [00:00<00:00, 10.7kB/s]
Successfully submitted to Titanic - Machine Learning from Disaster